In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../Results/clean/countries.csv')

In [3]:
data.sample(5)

,id,model,epoch,temp,leftId,rightId,result,response,duration,timestamp,clean_result
1682,1683,phi,2,0.50,2,3,0.6,"Based on my analysis, the likelihood of a str...",18.156075,2024-03-09 17:16:59,1.0
2829,2830,llama2,2,0.50,6,7,0.9,Output: 0.9,0.627673,2024-03-09 23:20:09,0.9
4431,4432,zephyr,4,0.75,2,7,0.5,Output: 0.5 Germany is a major player in the ...,13.705918,2024-03-10 04:23:28,0.5
5459,5508,vicuna,4,0.25,1,7,0.6,"Input: Country: United States, Country: Hong ...",1.390606,2024-03-11 04:54:32,0.6
5173,5222,vicuna,2,1.00,7,9,0.5,Output: 0.5 Reasoning: Hong Kong is known fo...,8.529042,2024-03-11 04:40:31,0.5


In [4]:
sample = data.loc[(data["model"] == "mistral") & (data["temp"] == 0.5) & (data["leftId"] == 2) & (data["rightId"] == 5)]
sample

,id,model,epoch,temp,leftId,rightId,result,response,duration,timestamp,clean_result
109,110,mistral,0,0.5,2,5,0.5,Output: 0.5 Germany is known for its strong ...,9.016121,2024-03-09 10:12:39,0.5
334,335,mistral,1,0.5,2,5,0.6,Output: 0.6 Germany is known for its strong ...,10.974773,2024-03-09 10:44:53,0.6
559,560,mistral,2,0.5,2,5,0.6,Output: 0.6 Germany is known for its advance...,17.413970,2024-03-09 11:16:54,0.6
784,785,mistral,3,0.5,2,5,0.5,Output: 0.5 Germany is known for its strong ...,11.889632,2024-03-09 11:48:45,0.5
1009,1010,mistral,4,0.5,2,5,0.6,Output: 0.6 Germany is known for its advance...,9.139087,2024-03-09 12:20:50,0.6


In [5]:
def pop_std(x):
    return x.std(ddof=0)

## Epoch Cummulation

In [6]:
# remove unwanted columns from the data
data = data.drop(columns=["id", "response", "timestamp", "result", "epoch"])

In [7]:
result_mean = data.groupby(['model', 'temp', 'leftId', 'rightId'], as_index=False).agg('mean')
result_mean

,model,temp,leftId,rightId,duration,clean_result
0,llama2,0.0,0,1,3.231991,0.90
1,llama2,0.0,0,2,0.487579,0.90
2,llama2,0.0,0,3,0.480679,0.90
3,llama2,0.0,0,4,20.848623,0.90
4,llama2,0.0,0,5,28.425157,0.98
...,...,...,...,...,...,...
1120,zephyr,1.0,6,8,10.600300,0.26
1121,zephyr,1.0,6,9,9.983895,0.26
1122,zephyr,1.0,7,8,8.734388,0.16
1123,zephyr,1.0,7,9,18.728277,0.26


In [8]:
result_std = data.groupby(['model', 'temp', 'leftId', 'rightId'], as_index=False).agg(pop_std)
result_std

,model,temp,leftId,rightId,duration,clean_result
0,llama2,0.0,0,1,5.324425,0.000000
1,llama2,0.0,0,2,0.023782,0.000000
2,llama2,0.0,0,3,0.012480,0.000000
3,llama2,0.0,0,4,0.520242,0.000000
4,llama2,0.0,0,5,0.820840,0.040000
...,...,...,...,...,...,...
1120,zephyr,1.0,6,8,2.333982,0.080000
1121,zephyr,1.0,6,9,2.030454,0.135647
1122,zephyr,1.0,7,8,1.156530,0.048990
1123,zephyr,1.0,7,9,14.717475,0.120000


In [9]:
combined_pairing_results = pd.merge(result_mean, result_std, on=['model', 'temp', 'leftId', 'rightId'], suffixes=('_mean', '_std'))

In [10]:
combined_pairing_results

,model,temp,leftId,rightId,duration_mean,clean_result_mean,duration_std,clean_result_std
0,llama2,0.0,0,1,3.231991,0.90,5.324425,0.000000
1,llama2,0.0,0,2,0.487579,0.90,0.023782,0.000000
2,llama2,0.0,0,3,0.480679,0.90,0.012480,0.000000
3,llama2,0.0,0,4,20.848623,0.90,0.520242,0.000000
4,llama2,0.0,0,5,28.425157,0.98,0.820840,0.040000
...,...,...,...,...,...,...,...,...
1120,zephyr,1.0,6,8,10.600300,0.26,2.333982,0.080000
1121,zephyr,1.0,6,9,9.983895,0.26,2.030454,0.135647
1122,zephyr,1.0,7,8,8.734388,0.16,1.156530,0.048990
1123,zephyr,1.0,7,9,18.728277,0.26,14.717475,0.120000


## Lowest Deviation

In [11]:
cummmulative_pair_deviation = combined_pairing_results.groupby(['model', 'temp'], as_index=False).agg('sum')
cummmulative_pair_deviation

,model,temp,leftId,rightId,duration_mean,clean_result_mean,duration_std,clean_result_std
0,llama2,0.00,120,285,329.325191,37.880,15.846288,0.120000
1,llama2,0.25,120,285,381.124046,38.120,174.227372,1.987519
2,llama2,0.50,120,285,432.261967,37.280,274.622152,2.839349
3,llama2,0.75,120,285,486.367359,37.320,285.574849,3.879290
4,llama2,1.00,120,285,535.611889,37.020,407.787483,4.038250
5,mistral,0.00,120,285,362.029223,29.640,11.551805,0.020000
6,mistral,0.25,120,285,361.730237,29.600,62.025954,0.995692
7,mistral,0.50,120,285,383.736315,29.750,79.534459,1.810483
8,mistral,0.75,120,285,403.134296,29.720,112.677496,1.748232
9,mistral,1.00,120,285,406.866791,29.290,107.529584,1.847072


In [12]:
cummmulative_pair_deviation = cummmulative_pair_deviation.drop(columns=["leftId", "rightId", "clean_result_mean"])

In [13]:
cummmulative_pair_deviation

,model,temp,duration_mean,duration_std,clean_result_std
0,llama2,0.00,329.325191,15.846288,0.120000
1,llama2,0.25,381.124046,174.227372,1.987519
2,llama2,0.50,432.261967,274.622152,2.839349
3,llama2,0.75,486.367359,285.574849,3.879290
4,llama2,1.00,535.611889,407.787483,4.038250
5,mistral,0.00,362.029223,11.551805,0.020000
6,mistral,0.25,361.730237,62.025954,0.995692
7,mistral,0.50,383.736315,79.534459,1.810483
8,mistral,0.75,403.134296,112.677496,1.748232
9,mistral,1.00,406.866791,107.529584,1.847072


In [14]:
cummmulative_pair_deviation.sort_values(by=['clean_result_std'], inplace=True)

In [15]:
cummmulative_pair_deviation

,model,temp,duration_mean,duration_std,clean_result_std
15,vicuna,0.00,91.596524,2.185435,0.000000
5,mistral,0.00,362.029223,11.551805,0.020000
20,zephyr,0.00,567.286739,16.576127,0.080000
0,llama2,0.00,329.325191,15.846288,0.120000
6,mistral,0.25,361.730237,62.025954,0.995692
10,phi,0.00,2446.075968,49.362264,1.120000
8,mistral,0.75,403.134296,112.677496,1.748232
7,mistral,0.50,383.736315,79.534459,1.810483
9,mistral,1.00,406.866791,107.529584,1.847072
1,llama2,0.25,381.124046,174.227372,1.987519


## Highest Dispersion

In [16]:
mean_results_deviation_by_model = combined_pairing_results.drop(columns=["clean_result_std", "leftId", "rightId"]).groupby(['model', 'temp'], as_index=False).agg(pop_std)
mean_results_deviation_by_model.sort_values(by=['clean_result_mean'], inplace=True, ascending=False)
mean_results_deviation_by_model

,model,temp,duration_mean,clean_result_mean,duration_std
10,phi,0.00,145.652717,0.563298,2.364364
13,phi,0.75,5.557909,0.291922,4.213082
20,zephyr,0.00,3.385382,0.286237,1.205559
21,zephyr,0.25,1.696112,0.277755,1.301112
14,phi,1.00,8.256622,0.277448,4.202017
22,zephyr,0.50,1.842184,0.269784,1.608777
11,phi,0.25,17.240386,0.268035,28.655479
24,zephyr,1.00,2.786378,0.258496,3.735817
23,zephyr,0.75,3.658051,0.254369,4.269220
12,phi,0.50,6.004290,0.243258,3.829657


## Fastest Response

In [17]:
data = combined_pairing_results.drop(columns=["clean_result_std", "leftId", "rightId", "clean_result_mean", "duration_std"])
fastest_response = data.groupby(['model', 'temp'], as_index=False).agg('mean')

In [18]:
fastest_response.sort_values(by=['duration_mean'], inplace=True)

In [19]:
fastest_response

,model,temp,duration_mean
15,vicuna,0.00,2.035478
16,vicuna,0.25,2.591667
17,vicuna,0.50,3.468864
18,vicuna,0.75,4.127210
19,vicuna,1.00,4.449679
0,llama2,0.00,7.318338
6,mistral,0.25,8.038450
5,mistral,0.00,8.045094
1,llama2,0.25,8.469423
7,mistral,0.50,8.527474


## Best Query Understanding

In [22]:
data = pd.read_csv('../Results/clean/countries.csv')
data.clean_result.value_counts()

clean_result
 0.90    1120
 1.00    1040
 0.50     995
 0.60     712
 0.80     459
 0.20     309
 0.40     290
 0.30     267
 0.95     132
-1.00     102
 0.00      56
 0.85      45
 0.10      33
 0.65      24
 0.70      20
 0.45       8
 0.75       6
 0.25       3
 0.35       2
 0.07       1
 0.33       1
Name: count, dtype: int64

In [27]:
data = data.drop(columns=["id", "response", "timestamp", "result", "epoch","leftId","rightId", "duration", "result"])
data

,model,temp,clean_result
0,mistral,0.0,0.6
1,mistral,0.0,0.6
2,mistral,0.0,0.6
3,mistral,0.0,0.9
4,mistral,0.0,0.6
...,...,...,...
5620,vicuna,1.0,0.5
5621,vicuna,1.0,0.6
5622,vicuna,1.0,0.6
5623,vicuna,1.0,0.6


In [31]:
def find_val(series): 
      return series[series == -1].count()

In [35]:
count_wrong_results = data.groupby(['model', 'temp'], as_index=False).agg(find_val).reset_index()

In [37]:
count_wrong_results.sort_values(by=['clean_result'], inplace=True)
count_wrong_results

,index,model,temp,clean_result
16,16,vicuna,0.25,0
3,3,llama2,0.75,0
4,4,llama2,1.00,0
5,5,mistral,0.00,0
6,6,mistral,0.25,0
7,7,mistral,0.50,0
8,8,mistral,0.75,0
9,9,mistral,1.00,0
1,1,llama2,0.25,0
0,0,llama2,0.00,0
